In [1]:
%load_ext autoreload

In [3]:
%matplotlib inline
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

import pandas as pd
import seaborn as sns
import numpy as np

import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split, KFold
from sklearn.grid_search import GridSearchCV

import transform_utils as tu
from kaggle_helper import KaggleHelper

In [4]:
kh = KaggleHelper("submissions.db", "predictions")

table submissions already exists
table validation_scores already exists


In [5]:
%aimport transform_utils
%aimport kaggle_helper

In [6]:
%aimport

Modules to reload:
kaggle_helper transform_utils

Modules to skip:



In [7]:
%autoreload

In [8]:
pd.read_csv("data/train.csv").head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000
1,30,7,Friday,60538815980,1,SHOES,8931
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017


In [37]:
start = datetime.datetime.now()

df = tu.preprocess_walmart(pd.read_csv("data/train.csv"))
df2 = tu.transform_column(df, 'DepartmentDescription', ['TripType', 'VisitNumber', 'ScanCount'])
df3 = tu.transform_group(df2, 'VisitNumber', True)

print("Total Time:", (datetime.datetime.now() - start).total_seconds())

Total Time: 4.210933


In [45]:
X = df3.drop("TripType", axis=1).drop("VisitNumber", axis=1)
sorted_columns = sorted(X.columns)
y = df3.TripType

In [39]:
XTrain, XTest, yTrain, yTest = train_test_split(X.values, y.values, test_size=0.2)
XTrain, XVal, yTrain, yVal = train_test_split(XTrain, yTrain, test_size=0.2)

http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html#sklearn.grid_search.GridSearchCV

In [40]:
# for x in [50, 75, 100]:
#     for train, test in KFold(len(a_train), 3):
#         clf = RandomForestClassifier(n_estimators=250, n_jobs=3, max_depth=x)
#         clf.fit(a_train[train], b_train[train])
#         print("max depth:", x, clf.score(a_train[test], b_train[test]))

In [41]:
start = datetime.datetime.now()

clf = RandomForestClassifier(n_jobs=3)
clf.fit(XTrain, yTrain)
score = clf.score(XVal, yVal)

print(score)
print("Total Time:", (datetime.datetime.now() - start).total_seconds())

0.637901750719
Total Time: 1.671383


In [42]:
#kh.save_validation_score(clf, score, len(XTrain), "")

# Only Kaggle Exists Below Here

In [48]:
kaggleTest = tu.preprocess_walmart(pd.read_csv("data/test.csv"))
test2 = tu.transform_column(kaggleTest, 'DepartmentDescription', ['VisitNumber', 'ScanCount'])
test3 = tu.transform_group(test2, 'VisitNumber')

In [49]:
missing_columns = set(df3.drop("TripType",axis=1).columns).difference(set(test3.columns))
for col in missing_columns:
    print(col)
    test3[col] = 0
test3 = test3[sorted_columns]

HEALTH AND BEAUTY AIDS


In [ ]:
test_predictions = clf.predict(test3.values)
output = tu.convert_predictions(test_predictions, 
                               output_index=pd.Series(test3.index), 
                               actual_trip_types=df.TripType.unique())

In [ ]:
kh.save_predictions(output, clf, len(a_train), "")